In [ ]:
# ==============================================================================
# LIBRARY INSTALLATION
# ==============================================================================
library(ggplot2)
library(dplyr)
library(jsonlite)
library(data.table)
library(stringr)

# ==============================================================================
# Load Parameters from JSON
# ==============================================================================
json_input <- fromJSON("galaxy_inputs/galaxy_inputs.json")

data_path         <- json_input$data$path                # Path to CSV or TSV input file
col_Y             <- json_input$colXY                    # Variable to plot on Y axis (e.g., "%Coverage")
col_group_line    <- json_input$colXgroupeXline          # Grouping variable for line (e.g., "site")
col_group_point   <- json_input$colXgroupeXpoint           # Grouping variable for point size (e.g., "sp richness")
plot_title        <- json_input$title                    # Title of the plot
x_axis_label      <- json_input$x                        # Label for X axis (usually "Year")
y_axis_label      <- json_input$y                        # Label for Y axis
plot_width        <- json_input$width                    # Width of output image in pixels
plot_res          <- json_input$res    
output_path       <- "outputs/plot.png"                  # Output plot path
ylim_values       <- json_input$ylim    
if (!is.null(json_input$ylim) && json_input$ylim != "") {ylim_values <- as.numeric(strsplit(ylim_values, " ")[[1]])}

# ==============================================================================
# Load Dataset
# ==============================================================================
df <- fread(data_path)

# ------------------------
# Clean Names (remove special characters like "%", ".", etc.)
# ------------------------
colnames(df) <- make.names(colnames(df))                
colnames(df) <- gsub("\\.", "", colnames(df))            

year_col <- colnames(df)[1]
y_col    <- gsub("\\.", "", make.names(col_Y))
group_line <- gsub("\\.", "", make.names(col_group_line))
group_point <- gsub("\\.", "", make.names(col_group_point))

# ==============================================================================
# Aggregate Data: Mean Y and Point Group per Year and Line Group
# ==============================================================================
if (!is.null(col_group_point)) {
  agg_df <- df %>%
    group_by(.data[[year_col]], .data[[group_line]]) %>%
    summarise(
      Y_value = mean(.data[[y_col]], na.rm = TRUE),
      Point_value = mean(.data[[group_point]], na.rm = TRUE),
      .groups = "drop"
    )
} else {
  agg_df <- df %>%
    group_by(.data[[year_col]], .data[[group_line]]) %>%
    summarise(
      Y_value = mean(.data[[y_col]], na.rm = TRUE),
      .groups = "drop"
    )
}


# ==============================================================================
# CREATE PLOT
# ==============================================================================
p <- ggplot(agg_df, aes(x = .data[[year_col]], y = Y_value, color = .data[[group_line]], group = .data[[group_line]])) +
  geom_line(linewidth = 1)

if (!is.null(col_group_point)) {
  p <- p + geom_point(aes(size = Point_value), alpha = 0.7) +
    labs(size = col_group_point)
}

if (!is.null(ylim_values)) {
  p <- p + ylim(ylim_values)
}

p <- p + labs(
    title = plot_title,
    x = x_axis_label,
    y = y_axis_label,
    color = col_group_line
  ) +
  theme_minimal()

png(output_path, width=plot_width, height=4, units="in", res=plot_res)
print(p)
dev.off()

